In [1]:
from datetime import datetime
from synthetix_v3.base_perps import TestBasePerps
import polars as pl

pl.Config.set_fmt_str_lengths(200)

polars.config.Config

In [2]:
base_perps = TestBasePerps()

In [5]:
# start and end blocks for the competition
start_block = 10536192
end_block = 11455992

start_timestamp = 1696260000
end_timestamp = 1698105600

In [6]:
settled_orders_df = base_perps.get_settled_orders(start_block=start_block, end_block=end_block) # https://goerli.basescan.org/block/10536192 # starting block time Monday, October 2, 2023 5:00:00 PM

In [7]:
polars_df = pl.from_pandas(settled_orders_df)

In [8]:
polars_df.columns

['orderSettleds_accountId',
 'orderSettleds_timestamp',
 'markets_id',
 'orderSettleds_fillPrice',
 'orderSettleds_accruedFunding',
 'orderSettleds_sizeDelta',
 'orderSettleds_newSize',
 'orderSettleds_totalFees',
 'orderSettleds_referralFees',
 'orderSettleds_collectedFees',
 'orderSettleds_settlementReward',
 'markets_marketSymbol']

In [9]:
accounts_df_polars = pl.from_pandas(acct_df)

In [10]:
acct_orders_df = polars_df.join(
    other=accounts_df_polars.select('accounts_accountId', 'accounts_owner'), 
    left_on='orderSettleds_accountId', 
    right_on='accounts_accountId', 
    how='left')

In [11]:
cleaned_settled_orders = (acct_orders_df.select([
    pl.col('orderSettleds_timestamp'),
    pl.col('markets_marketSymbol'),
    pl.col('orderSettleds_accountId').cast(str),
    pl.col('accounts_owner'),
    pl.col('orderSettleds_fillPrice'),
    pl.col('orderSettleds_accruedFunding'),
    pl.col('orderSettleds_sizeDelta'),
    pl.col('orderSettleds_newSize'),
    pl.col('orderSettleds_totalFees')
])
.with_columns([
    pl.from_epoch("orderSettleds_timestamp").alias("datetime"),
    (pl.col('orderSettleds_fillPrice') / 10 ** 18),
    (pl.col('orderSettleds_sizeDelta') / 10 ** 18),
    (pl.col('orderSettleds_accruedFunding') / 10 ** 18),
    (pl.col('orderSettleds_newSize') / 10 ** 18),
    (pl.col('orderSettleds_totalFees') / 10 ** 18),
])
.with_columns([
    (pl.col('orderSettleds_sizeDelta').abs() * pl.col('orderSettleds_fillPrice')).alias('size_usd')
]))

In [12]:
cleaned_settled_orders.select(pl.sum('size_usd'))

size_usd
f64
2.8033e9


### Liquidations

In [16]:
market_price_df = base_perps.get_market_updates()

In [17]:
pos_liqs_df = base_perps.get_position_liquidations(start_block=start_block, end_block=end_block)

In [18]:
position_liquidations_df = pl.from_pandas(pos_liqs_df)

In [19]:
# normalize numbers
market_price_polars_df = pl.from_pandas(market_price_df).with_columns([
    pl.from_epoch('marketUpdateds_timestamp').dt.round("1s").alias('datetime'),
    pl.col('marketUpdateds_price') / 10**18,
    pl.col('marketUpdateds_skew') / 10**18,
    pl.col('marketUpdateds_size') / 10**18,
    pl.col('marketUpdateds_sizeDelta') / 10**18,
    pl.col('marketUpdateds_currentFundingRate') / 10**18,
    pl.col('marketUpdateds_currentFundingVelocity') / 10**18
])

# normalize numbers
position_liquidations_df = position_liquidations_df.with_columns([
    pl.from_epoch('positionLiquidateds_timestamp').dt.round("1s").alias('datetime'),
    pl.col('positionLiquidateds_amountLiquidated') / 10**18,
    pl.col('positionLiquidateds_currentPositionSize') / 10**18,
    ]).filter(pl.col("datetime") > datetime(2023, 10, 2))

In [20]:
position_liquidations_usd_df = position_liquidations_df.join(market_price_polars_df, on='datetime', how='left')

In [21]:
position_liquidations_usd_df.head(5)

positionLiquidateds_id,positionLiquidateds_timestamp,positionLiquidateds_accountId,markets_id,positionLiquidateds_amountLiquidated,positionLiquidateds_currentPositionSize,markets_marketSymbol,datetime,marketUpdateds_timestamp,markets_id_right,marketUpdateds_price,marketUpdateds_skew,marketUpdateds_size,marketUpdateds_sizeDelta,marketUpdateds_currentFundingRate,marketUpdateds_currentFundingVelocity,markets_marketSymbol_right
str,i64,str,i64,f64,f64,str,datetime[μs],i64,i64,f64,f64,f64,f64,f64,f64,str
"""300-170141183460469231731687303715884105856-11453845""",1698101306,"""170141183460469231731687303715884105856""",300,1234.3639,0.0,"""LINK""",2023-10-23 22:48:26,1698101306,300,10.533,137791.810257,151142.499457,1234.3639,-0.077125,0.248025,"""LINK"""
"""300-170141183460469231731687303715884105856-11453825""",1698101266,"""170141183460469231731687303715884105856""",300,19999.8,-1234.3639,"""LINK""",2023-10-23 22:47:46,1698101266,300,10.526109,136557.446357,152376.863357,19999.8,-0.077238,0.245803,"""LINK"""
"""300-170141183460469231731687303715884105856-11453804""",1698101224,"""170141183460469231731687303715884105856""",300,19999.8,-21234.1639,"""LINK""",2023-10-23 22:47:04,1698101224,300,10.56526,116557.646357,172376.663357,19999.8,-0.07734,0.209804,"""LINK"""
"""300-170141183460469231731687303715884105856-11453784""",1698101184,"""170141183460469231731687303715884105856""",300,19999.8,-41233.9639,"""LINK""",2023-10-23 22:46:24,1698101184,300,10.528384,96557.846357,192376.463357,19999.8,-0.077421,0.173804,"""LINK"""
"""300-170141183460469231731687303715884105856-11453764""",1698101144,"""170141183460469231731687303715884105856""",300,19999.8,-61233.7639,"""LINK""",2023-10-23 22:45:44,1698101144,300,10.632118,76558.046357,212376.263357,19999.8,-0.077485,0.137804,"""LINK"""


In [22]:
position_liquidations_usd_df.columns

['positionLiquidateds_id',
 'positionLiquidateds_timestamp',
 'positionLiquidateds_accountId',
 'markets_id',
 'positionLiquidateds_amountLiquidated',
 'positionLiquidateds_currentPositionSize',
 'markets_marketSymbol',
 'datetime',
 'marketUpdateds_timestamp',
 'markets_id_right',
 'marketUpdateds_price',
 'marketUpdateds_skew',
 'marketUpdateds_size',
 'marketUpdateds_sizeDelta',
 'marketUpdateds_currentFundingRate',
 'marketUpdateds_currentFundingVelocity',
 'markets_marketSymbol_right']

In [23]:
cleaned_settled_orders.columns

['orderSettleds_timestamp',
 'markets_marketSymbol',
 'orderSettleds_accountId',
 'accounts_owner',
 'orderSettleds_fillPrice',
 'orderSettleds_accruedFunding',
 'orderSettleds_sizeDelta',
 'orderSettleds_newSize',
 'orderSettleds_totalFees',
 'datetime',
 'size_usd']

In [28]:
# rename columns for concatting
position_liq_final = position_liquidations_usd_df.rename({
    'positionLiquidateds_accountId': 'accountId', 
    'markets_marketSymbol_right': 'marketSymbol',
    'positionLiquidateds_amountLiquidated': 'size_delta',
    'positionLiquidateds_currentPositionSize': 'size',
    'marketUpdateds_price': 'price'
     }).with_columns([
    pl.lit(True).alias('liquidation')
]).select('datetime', 'accountId', 'marketSymbol', 'size_delta', 'size', 'price', 'liquidation')

settled_order_final = cleaned_settled_orders.rename({
    'orderSettleds_accountId': 'accountId', 
    'markets_marketSymbol': 'marketSymbol',
    'orderSettleds_sizeDelta': 'size_delta',
    'orderSettleds_newSize': 'size',
    'orderSettleds_fillPrice': 'price',
     }).with_columns([
    pl.lit(False).alias('liquidation')]).select('datetime', 'accountId', 'marketSymbol', 'size_delta', 'size', 'price', 'liquidation')

In [29]:
# get the overlap between unique accuonts
unique_liq_accts = position_liq_final['accountId'].unique().to_list()
unique_orders_accts = settled_order_final['accountId'].unique().to_list()

print(len(set(unique_liq_accts)))
print(len(set(unique_orders_accts)))

84
131


In [30]:
# are there liquidation accounts that are not in unique order accts?
print(len(set(unique_orders_accts) - set(unique_liq_accts)))

51


### Concat data, join account owners, and analyze results

In [ ]:
acct_df = base_perps.get_accounts()

In [32]:
concat_df = pl.concat([position_liq_final, settled_order_final]).join(
    other=accounts_df_polars.select('accounts_accountId', 'accounts_owner'), 
    left_on='accountId', 
    right_on='accounts_accountId', 
    how='left')

In [34]:
concat_df.write_parquet('../perps_v3_competition/perps_v3_trades.parquet')

In [ ]:
concat_df.filter(pl.col('liquidation') == False).group_by('accountId').agg([pl.count()]).sort(by='count', descending=True)

accountId,count
str,u32
"""170141183460469231731687303715884105740""",167
"""170141183460469231731687303715884105797""",115
"""170141183460469231731687303715884105769""",106
"""170141183460469231731687303715884105803""",101
"""170141183460469231731687303715884105794""",100
"""170141183460469231731687303715884105780""",90
"""170141183460469231731687303715884105863""",83
"""170141183460469231731687303715884105773""",81
"""89029""",67


In [ ]:
concat_df.filter(pl.col('liquidation') == True).shape

(441, 7)

In [ ]:
concat_df.filter(pl.col('liquidation') == False).shape

(2287, 7)

In [ ]:
filtered_acct = concat_df['accountId'][0]

In [ ]:
concat_df.filter(pl.col('accountId') == filtered_acct).filter(pl.col('marketSymbol') == 'ETH').sort(by='datetime', descending=False).tail(20)

datetime,accountId,marketSymbol,size_delta,size,price,liquidation
datetime[μs],str,str,f64,f64,f64,bool
2023-10-08 07:12:10,"""170141183460469231731687303715884105856""","""ETH""",184.0022,184.0022,1632.097314,false
2023-10-23 22:43:26,"""170141183460469231731687303715884105856""","""ETH""",19999.8,-121233.1639,1751.600241,true
2023-10-23 22:43:26,"""170141183460469231731687303715884105856""","""ETH""",184.0022,0.0,1751.600241,true
